In [26]:
import sys
sys.path.append('../')
from utility.file_utility import FileUtility
from scipy.sparse import csr_matrix
from shutil import copyfile



In [27]:
labeler=dict([tuple(x.split(',')) for x in FileUtility.load_list('../../bio_cpe_datacollection/representations/RA/grouping.csv')[1::]])

In [28]:
SRX2row=dict([(x.split('/')[-1].split('.')[0],idx) for idx, x in enumerate(FileUtility.load_list('../../bio_cpe_datacollection/representations/RA/RA_100000_cpe_10000_meta'))])

In [5]:
def get_matrix_label(X, labeler, SRX2row, pos=[], neg=[]):
    label_vec=[]
    X=X.toarray()
    New_X=[]
    for srx,label in labeler.items():
        if label in pos:
            New_X.append(X[SRX2row[srx],:])
            label_vec.append(1)
        elif label in neg:
            New_X.append(X[SRX2row[srx],:])
            label_vec.append(0)
    return csr_matrix(New_X), label_vec
    

In [29]:
mat=FileUtility.load_sparse_csr('../../16S_datasets/ra/rep/ra_selfposcpe_50000_cpe_5000.npz')
features=FileUtility.load_list('../../16S_datasets/ra/rep/ra_selfposcpe_50000_cpe_5000_features')


In [30]:
X,labels=get_matrix_label(mat, labeler, SRX2row, pos=['untreated_RA'], neg=['healthy','psoriatic', 'treated_RA'])

In [37]:
__author__ = "Ehsaneddin Asgari"
__license__ = "GPL"
__version__ = "1.0.0"
__maintainer__ = "Ehsaneddin Asgari"
__email__ = "asgari@berkeley.edu or ehsaneddin.asgari@helmholtz-hzi.de"
__project__ = "LLP - Life Language Processing"
__website__ = "https://llp.berkeley.edu/"

import sys
sys.path.append('../')
from utility.file_utility import FileUtility
from utility.list_set_util import get_max_of_dict
import numpy as np
from chi2analysis.chi2analysis import Chi2Analysis


class Seq2Tax(object):

    def __init__(self):
        self.seq2tax=FileUtility.load_obj('/mounts/data/proj/asgari/dissertation/git_repos/16S_datasets/GG/to_be_checked/seq2tax.pickle')


    def extract_markers(self, path, X, Y, features):
        '''
        :param path:
        :param X:
        :param Y:
        :param features:
        :return:
        '''
        CA=Chi2Analysis(X, Y, features)
        res_bin=CA.extract_features_fdr(path+'_chi2_binary.txt',1000,direction=True,binarization=True)
        res_med=CA.extract_features_fdr(path+'_chi2_median.txt',1000,direction=True,binarization='median')
        pos_bin,neg_bin=self.extract_top_but_n(res_bin,1000)
        pos_med,neg_med=self.extract_top_but_n(res_med,1000)
        Seq2Tax.write_in_fastafile(path+'_chi2_binary.fasta',res_bin)
        Seq2Tax.write_in_fastafile(path+'_chi2_relative.fasta',res_med)
        Seq2Tax.write_in_file(path+'_chi2_binary_taxa.txt',pos_bin,neg_bin)
        Seq2Tax.write_in_file(path+'_chi2_median_taxa.txt',pos_med,neg_med)

    @staticmethod
    def write_in_fastafile(filename,res, min_length=50):
        corpus=[]
        labels=[]
        for seq, score, pval, _, _ in res:
            if len(seq)>min_length:
                corpus.append(seq)
                labels.append(' '.join(['+' if score>0 else '-','p-val:'+str(pval)]))
        FileUtility.create_fasta_file(filename,corpus,labels)
        
    @staticmethod
    def write_in_file(filename,pos,neg):
        lines=[['direction','marker','p-value','genus']]
        for marker, pval,genus in pos:
            lines.append(['+',marker, str(pval),genus])
        for marker, pval,genus in neg:
            lines.append(['-',marker, str(pval),genus])
        FileUtility.save_list(filename, ['\t'.join(line) for line in lines])

    def micro_seq_to_tax(self, seq):
        tax_dict=self.seq2tax[seq]
        result_dict=dict()
        for level,level_details in tax_dict.items():
                    pairs=[(pairs.split('###')[0],float(pairs.split('###')[1]))for pairs in level_details.split()]
                    for pair in pairs:
                        (tax, prob)=pair
                        if prob>0.8:
                            if level not in result_dict:
                                result_dict[level]=dict()
                            if tax not in result_dict[level]:
                                result_dict[level][tax]=0
                            result_dict[level][tax]+=prob
        if 'species' in result_dict:
            return result_dict['species']
        elif 'genus' in result_dict:
            return result_dict['genus']
        elif 'family' in result_dict:
            return result_dict['family']
        elif 'order' in result_dict:
            return result_dict['order']
        elif 'class' in result_dict:
            return result_dict['class']
        elif 'phylum' in result_dict:
            return result_dict['phylum']
        else:
            return -1

    def extract_top_but_n(self, chi2_res, n, threshold=3.8):
        # 5
        pos=[]
        neg=[]
        idx=0
        while len(pos)<n and idx < len(chi2_res):
            if chi2_res[idx][1]>threshold  and chi2_res[idx][0] in self.seq2tax:
                if 'genus' in self.seq2tax[chi2_res[idx][0]]:
                    dict_tax={pairs.split('###')[0]:float(pairs.split('###')[1]) for pairs in self.seq2tax[chi2_res[idx][0]]['genus'].split()}
                    if np.max(list(dict_tax.values())) > 0.95:
                        pos.append((chi2_res[idx][0], chi2_res[idx][2], get_max_of_dict(dict_tax)))
                if 'family' in self.seq2tax[chi2_res[idx][0]]:
                    dict_tax={pairs.split('###')[0]:float(pairs.split('###')[1]) for pairs in self.seq2tax[chi2_res[idx][0]]['family'].split()}
                    if np.max(list(dict_tax.values())) > 0.95:
                        pos.append((chi2_res[idx][0], chi2_res[idx][2], get_max_of_dict(dict_tax)))
            idx+=1
        idx=0
        while len(neg)<n and idx < len(chi2_res):
            if chi2_res[idx][1]<-threshold and idx< len(chi2_res) and chi2_res[idx][0] in self.seq2tax:
                if 'genus' in self.seq2tax[chi2_res[idx][0]]:
                    dict_tax={pairs.split('###')[0]:float(pairs.split('###')[1]) for pairs in self.seq2tax[chi2_res[idx][0]]['genus'].split()}
                    if np.max(list(dict_tax.values())) > 0.95:
                        neg.append((chi2_res[idx][0], chi2_res[idx][2], get_max_of_dict(dict_tax)))
                elif 'family' in self.seq2tax[chi2_res[idx][0]]:
                    dict_tax={pairs.split('###')[0]:float(pairs.split('###')[1]) for pairs in self.seq2tax[chi2_res[idx][0]]['family'].split()}
                    if np.max(list(dict_tax.values())) > 0.95:
                        neg.append((chi2_res[idx][0], chi2_res[idx][2], get_max_of_dict(dict_tax)))
            idx+=1
        return pos, neg




In [38]:
SeqTaxa=Seq2Tax()

In [39]:
SeqTaxa.extract_markers('/mounts/data/proj/asgari/dissertation/git_repos/16Seq2Seg/final_results/RA/markers/self_cpe_d_vs_all_',X,labels,features)

/mounts/Users/student/asgari/.local/lib/python3.4/site-packages/sklearn/feature_selection/univariate_selection.py:619: RuntimeWarning: invalid value encountered in less_equal
  np.arange(1, n_features + 1)]
/mounts/Users/student/asgari/.local/lib/python3.4/site-packages/sklearn/feature_selection/univariate_selection.py:622: RuntimeWarning: invalid value encountered in less_equal
  return self.pvalues_ <= selected.max()
